# Walkthrough

### Using Pint

LEOrbit uses the [`pint`](https://pint.readthedocs.io/en/0.10.1/index.html) library to handle physical quantities. It handles units and dimensions in a very practical way and prevent any units mistakes.

If you do not know `pint`, we suggest you take 5min to take a look at the main features of this library

In [1]:
from leorbit import Q_ # We import the `Quantity` object that uses the library's units registry

Q_("3.4s"), Q_("3hour+43min+33s"), Q_("3.45km")

(3.4 <Unit('second')>,
 3.7258333333333336 <Unit('hour')>,
 3.45 <Unit('kilometer')>)

### Getting GP data (orbital elements)

The first thing to do when one wants to make satellite predictions, is to gather the latest orbital elements, also known as GP (Ground Perturbations) data.
To do this, we use [celestrag.org](https://celestrak.org) and their API.

Using LEOrbit, there are currently 3 ways to create orbital elements:

* From GP data that you manually fetched on Celestrak
* From state vectors at given epoch
* ~~From a TLE (Two Line Elements)~~ outdated format...
* Automatically with just the satellite's NORAD catalog ID

##### From existant GP data dictionary

If you already have the GP data that you want to use (in Celestrak's format), you can easily create a `OrbitalElements` out of it:

In [2]:
from leorbit import OrbitalElements

gp_data_celestrak = { # NOTE: good GP for this walkthrough
    "OBJECT_NAME": "ISS (ZARYA)",
    "OBJECT_ID": "1998-067A",
    "EPOCH": "2024-03-27T12:13:52.819968",
    "MEAN_MOTION": 15.49609731,
    "ECCENTRICITY": 0.0004676,
    "INCLINATION": 51.6411,
    "RA_OF_ASC_NODE": 0.3077,
    "ARG_OF_PERICENTER": 8.9972,
    "MEAN_ANOMALY": 162.1343,
    "EPHEMERIS_TYPE": 0,
    "CLASSIFICATION_TYPE": "U",
    "NORAD_CAT_ID": 25544,
    "ELEMENT_SET_NO": 999,
    "REV_AT_EPOCH": 44587,
    "BSTAR": 0.0006325,
    "MEAN_MOTION_DOT": 0.00035317,
    "MEAN_MOTION_DDOT": 0
}

gp_data_iss = OrbitalElements.from_celestrak_json(gp_data_celestrak)
print(gp_data_iss)

OrbitalElements(epoch=<Time unixepoch=1711541632.819968>, eccentricity=<Quantity(0.0004676, 'dimensionless')>, inclination=<Quantity(51.6411, 'degree')>, ra_of_asc_node=<Quantity(0.3077, 'degree')>, arg_of_pericenter=<Quantity(8.9972, 'degree')>, mean_motion=<Quantity(15.4960973, 'turn / day')>, mean_anomaly=<Quantity(162.1343, 'degree')>, mean_motion_dot=<Quantity(0.00070634, 'turn / day ** 2')>, mean_motion_ddot=<Quantity(0, 'turn / day ** 3')>, bstar=<Quantity(0.0006325, '1 / earthRadii')>, name='ISS (ZARYA)', norad_cat_id=25544, eccentric_anomaly=<Quantity(162.142516, 'degree')>, true_anomaly=2.830064112989053, semi_major_axis=<Quantity(6796003.88, 'meter')>, semi_minor_axis=<Quantity(6796003.14, 'meter')>, time_at_periaster=<Time unixepoch=1711539121.7211266>)


#### From state vectors

You may have a state vector (aka. position and velocity in a given frame) of your satellite at a given time, that you want to propagate.

You can indeed create a `OrbitalElements` object out of it, but **be careful,** doing so discards any information about *drag* and the resulting propagation will be prone to errors quickly!

This way of creating orbital elements is therefore not recommended, and for users that know what they are doing.

In [3]:
from leorbit import Coordinates, Vec3, Time, AbsoluteFrame

t0 = Time.fromisoformat("2024-03-27T14:52:11.348683+00:00")
pos_gcrf = Vec3(x=Q_(2902093, "m"), y=Q_(3802436, "m"), z=Q_(4816229, "m"))
vel_gcrf = Vec3(x=Q_(-6930, "m/s"), y=Q_(2049, "m/s"), z=Q_(2555, "m/s"))
c = Coordinates(AbsoluteFrame.GCRF, pos_gcrf, vel_gcrf, t0)

gp_data_iss = OrbitalElements.from_state_vectors(c)
print(gp_data_iss)

OrbitalElements(epoch=<Time unixepoch=1711551131.348683>, eccentricity=<Quantity(0.00058293828, 'dimensionless')>, inclination=<Quantity(0.901098337, 'radian')>, ra_of_asc_node=<Quantity(6.27942891, 'radian')>, arg_of_pericenter=<Quantity(1.64322154, 'radian')>, mean_motion=<Quantity(0.00112804511, 'radian / second')>, mean_anomaly=<Quantity(5.77189767, 'radian')>, mean_motion_dot=<Quantity(0.0, 'radian / second ** 2')>, mean_motion_ddot=<Quantity(0.0, 'radian / second ** 3')>, bstar=<Quantity(0.0, '1 / meter')>, name='No name', norad_cat_id=None, eccentric_anomaly=<Quantity(5.77161229, 'radian')>, true_anomaly=-0.5118584642539967, semi_major_axis=<Quantity(6791436.02, 'meter')>, semi_minor_axis=<Quantity(6791434.86, 'meter')>, time_at_periaster=<Time unixepoch=1711546014.6227539>)


#### From NORAD catalog ID

The fastest and easiest way: LEOrbit and fetch automatically the latest GP data of a satellite on Celestrak just with its NORAD catalog ID:

In [4]:
from leorbit import OrbitalElements

gp_data_iss = OrbitalElements.from_celestrak_norad_cat_id(25544)
print(gp_data_iss)

OrbitalElements(epoch=<Time unixepoch=1711600276.89168>, eccentricity=<Quantity(0.0004611, 'dimensionless')>, inclination=<Quantity(51.6398, 'degree')>, ra_of_asc_node=<Quantity(356.9526, 'degree')>, arg_of_pericenter=<Quantity(12.1144, 'degree')>, mean_motion=<Quantity(15.4963634, 'turn / day')>, mean_anomaly=<Quantity(347.9955, 'degree')>, mean_motion_dot=<Quantity(0.00054178, 'turn / day ** 2')>, mean_motion_ddot=<Quantity(0, 'turn / day ** 3')>, bstar=<Quantity(0.00048707, '1 / earthRadii')>, name='ISS (ZARYA)', norad_cat_id=25544, eccentric_anomaly=<Quantity(347.990003, 'degree')>, true_anomaly=-0.2097099652554939, semi_major_axis=<Quantity(6795926.08, 'meter')>, semi_minor_axis=<Quantity(6795925.36, 'meter')>, time_at_periaster=<Time unixepoch=1711594887.309764>)


*Info: LEOrbit fetches GP data on Celestrak's API every hours, and stores locally the last fetched data, to avoid API's spam.*

### Create a propagator

With the latest GP data, we can now create a propagator to propagate in time the orbital elements. There are currently two propagators in LEOrbit:
* `SGP4`: most used propagator for satellite propagation, good accuracy with inclusion of perturbations due to drag and more.
* the `NoPropagator` propagator: it just propagates the mean anomaly $ M $ on the given ellipsis without perturbating it. You may find uses for it

In [5]:
from leorbit import SGP4

iss_sgp4 = SGP4(gp_data_iss)

And this would be enough to already find coordinates at later time of ISS. 
LEOrbit also implements a class to handle time in a very efficient way:

In [6]:
now = Time.now()
iss_sgp4.propagate(now)

<Coordinates: epoch=2024-03-28 at 11:08:53, Vec3(x=-1039568.6684180843 meter, y=4221981.141827323 meter, z=5213746.451572311 meter) in AbsoluteFrame.GCRF>

But you would rather use the `Satellite` class wrapper that makes it more elegant to compute coordinates

In [7]:
from leorbit import Satellite

iss = Satellite(iss_sgp4, "ISS", 25544)
print(iss)

<Satellite name='ISS' t₀='2024-03-28 at 04:31:16' [SGP4]>


### The wrapper that does everything automatically: `get_sat`

As you can see, LEOrbit allows you to manually create a `OrbitalElements` object, makes you choose your `Propagator` and then you have to instantiate the `Satellite` class.

But in reality, you'd only want to do this manually for specific use cases, and 95% of the time, you will always:

* Know your satellite's NORAD catalog ID
* Want to gather the **latest** GP for your satellite
* Want to use the SGP4 propagator

Therefore, you will want to use the `get_sat` function that takes your satellite's ID as an argument, and returns the corresponding `Satellite` object with the latest GP data, and the SGP4 propagator

In [8]:
from leorbit import get_sat

iss: Satellite = get_sat(25544)
print(iss)

<Satellite name='ISS (ZARYA)' t₀='2024-03-28 at 04:31:16' [SGP4]>


Now, you can find the coordinates of your satellite at any time!

In [9]:
iss.coordinates(now + Q_("3hour + 12min + 33s"))

<Coordinates: epoch=2024-03-28 at 14:21:26, Vec3(x=-3877028.346550973 meter, y=3677847.4402091783 meter, z=4192135.9582483764 meter) in AbsoluteFrame.GCRF>

## Coordinates and frames transformations

One of the best features of LEOrbit is easy coordinates conversions from/to any frames.

Many other Python libraries would return, as a result of the propagation of your GP data, a tuple or a list with floats, that you would just guess their units and meaning.

With LEOrbit, when you request your satellite's position at any time, you get a `Coordinate` object:

In [10]:
t: Time = now + Q_("2 day + 18 hour + 49 seconds")
c: Coordinates = iss.coordinates(t)
print(c)

<Coordinates: epoch=2024-03-31 at 05:09:42, Vec3(x=5435610.979107603 meter, y=-3506597.9150513527 meter, z=-2084100.6981637124 meter) in AbsoluteFrame.GCRF>


And now what if you want to retrieve the vector corresponding to the position at the given time, in GCRF, or ITRF? GPS?

In [11]:
c.to_gcrf()

Vec3(x=5435610.979107603 meter, y=-3506597.9150513527 meter, z=-2084100.6981637124 meter)

In [12]:
c.to_itrf()

Vec3(x=3168120.2269536434 meter, y=5639601.925810439 meter, z=-2084100.6981637124 meter)

In [13]:
c.to_gps()

GPSCoordinates(longitude_deg=60.67432993930043, latitude_deg=-17.858418750188044)

As simple as that!

The `Coordinates` object contains all the information about the coordinates, as it should be. Vectors are just a projection of those coordinates in a particular frame.

It makes more sense in a physical point of view, and the implementation is very *Pythonesque.*

### "Absolute" and "Relative" frames

LEOrbit handles the `GCRF (Geocentric Celestial Reference Frame)` and `ITRF (International Terrestrial Reference Frame)`, the most useful frames for LEO (Low Earth Orbit). 

Their are called `AbsoluteFrame` in LEOrbit since their are well defined from physical entities (like Earth and Sun).

The user can also create its own frames, and those would be `RelativeFrame` since they would be defined from an `AbsoluteFrame`.

Let's create a relative frame from a location on Earth, Paris (France) for example:

In [14]:
from leorbit import GPSCoordinates

gps_paris = GPSCoordinates(2.333333, 48.866667, 0., "Paris (FR)")
frame_paris = gps_paris.earth_local_frame()
print(frame_paris)

<EarthLocalFrame at GPS location 48.0° 52.0′ 0.0″ N, 2.0° 19.0′ 59.0″ E>


You can now get the projection of your previous coordinates in this relative frame:

In [15]:
c.pos_in_frame(frame_paris)

Vec3(x=-3928073.167027774 meter, y=5505942.054327796 meter, z=-5703129.055466725 meter)

*See the definition of `EarthLocalFrame` axises*

The most use for a `EarthLocalFrame` is the [horizontal coordinates](https://en.wikipedia.org/wiki/Horizontal_coordinate_system). Using a `EarthLocalFrame` or even just a `GPSCoordinates`, you can get the projection of any positon in Horizontal Coordinates (at given location):

In [16]:
c.to_horizontal_coordinates(gps_paris)

HorizontalCoordinates(azimuth_deg=125.50498533980455, altitude_deg=-40.13823963903124)

*NB: if the 'altitude' is negative, it means the satellite is not visible in your sky*

## Make a simulation

You have all the tools to generate a full trajectory of your satellite over a given period of time!

Let's make a "CSV like" file to illustrate:

In [17]:
from leorbit import Timeline

tl = Timeline(start=now, stop=now + Q_("1 day"), dt=Q_("1h")) # You would rather use something like dt=5s for real use cases
cs: list[Coordinates] = iss.simulation(tl)
print(f"              Time in ISO format	X (ITRF)	Y (ITRF)	Z (ITRF)")
for c in cs:
    p = c.to_itrf()
    print(f"{c.epoch.isoformat}	{p.x.m_as('m'):09.0f}	{p.y.m_as('m'):09.0f}	{p.z.m_as('m'):09.0f}")

              Time in ISO format	X (ITRF)	Y (ITRF)	Z (ITRF)
2024-03-28T11:08:53.268085+00:00	-01504869	004079362	005213746
2024-03-28T12:08:53.268085+00:00	005520451	-03201648	-02337694
2024-03-28T13:08:53.268085+00:00	-06244536	001224509	-02402526
2024-03-28T14:08:53.268085+00:00	003770973	002123581	005228977
2024-03-28T15:08:53.268085+00:00	-00363206	-05504290	-03973523
2024-03-28T16:08:53.268085+00:00	-02168462	006430828	-00429074
2024-03-28T17:08:53.268085+00:00	003571590	-03655169	004467672
2024-03-28T18:08:53.268085+00:00	-04426776	-01197855	-05020880
2024-03-28T19:08:53.268085+00:00	004341126	004977249	001607660
2024-03-28T20:08:53.268085+00:00	-02257482	-05634322	003042575
2024-03-28T21:08:53.268085+00:00	-01819174	003818041	-05326493
2024-03-28T22:08:53.268085+00:00	005727144	-01325721	003405589
2024-03-28T23:08:53.268085+00:00	-06642026	-00810773	001165291
2024-03-29T00:08:53.268085+00:00	003796258	002893388	-04846335
2024-03-29T01:08:53.268085+00:00	000781026	-04840335	00469

*NB: each call to `coordinates` is stored in memory using `lru_cache`, so that once a coordinate at given epoch is computed, recalling the function with the same epoch does not recompute everything*

## Events computation

You may need to access certain information over a simulation, such as:
* When is it night time at a given location on Earth?
* When is the Moon visible from the satellite?
* When is the satellite visible from a given location on Earth?

To answer those question, you can use `AstroEvent` objects.

In LEOrbit, an event is a class inheriting from the `AstroEvent` parent class.
They can have any arguments in their initializer.
They implement a `predicate(self, epoch: Time) -> bool` method, that returns `True` if the condition is met at the given epoch, aka if the event occurs at given epoch.

And most importantly, they implement the `compute_intervals(self, over: Timeline) -> list[Timeline]` method that returns all the time intervals when the event is met.

LEOrbit has a few buil-in events, and you can very easily create custom events!

For this example, let's say you want to know when the ISS is visible from Paris. You can do:

In [18]:
from leorbit import VisibleFromLocation

# Event corresponding to the visibility of ISS from Paris
visi_event = VisibleFromLocation(iss, gps_paris) 

# When will the ISS be visible from Paris in the next 5 days?
tl = Timeline(now, now + Q_("5 day"), Q_("5s"))
intervals: list[Timeline] = visi_event.compute_intervals(tl)
[interval.human for interval in intervals]


["Timeline: from '2024-03-28 at 14:01:33' to '2024-03-28 at 14:07:28', duration=5 min 55 s",
 "Timeline: from '2024-03-28 at 15:37:43' to '2024-03-28 at 15:44:28', duration=6 min 45 s",
 "Timeline: from '2024-03-28 at 17:14:48' to '2024-03-28 at 17:21:23', duration=6 min 35 s",
 "Timeline: from '2024-03-28 at 18:51:38' to '2024-03-28 at 18:58:23', duration=6 min 45 s",
 "Timeline: from '2024-03-28 at 20:29:03' to '2024-03-28 at 20:33:43', duration=4 min 40 s",
 "Timeline: from '2024-03-29 at 13:14:08' to '2024-03-29 at 13:19:03', duration=4 min 55 s",
 "Timeline: from '2024-03-29 at 14:49:38' to '2024-03-29 at 14:56:23', duration=6 min 45 s",
 "Timeline: from '2024-03-29 at 16:26:38' to '2024-03-29 at 16:33:13', duration=6 min 35 s",
 "Timeline: from '2024-03-29 at 18:03:33' to '2024-03-29 at 18:10:18', duration=6 min 45 s",
 "Timeline: from '2024-03-29 at 19:40:38' to '2024-03-29 at 19:46:18', duration=5 min 40 s",
 "Timeline: from '2024-03-30 at 12:27:03' to '2024-03-30 at 12:30:18',

But sometimes, a single event is not enough, and you would like to know when two (or more) events are happening at the same time.

For this example, let's say you want to know when the ISS is visible from Paris, AND when it is night time in Paris (so that you could try to observe it, for instance).

`AstroEvent` objects implement **intersections** and **unions** for this purpose:

In [19]:
from leorbit import AstroEvent, NightTime

# Event corresponding to night time in Paris
night_event = NightTime(gps_paris)

# This event corresponds to when the ISS is visible from paris AND when it is night time in Paris!
visi_and_night_event: AstroEvent = visi_event & night_event
intervals: list[Timeline] = visi_and_night_event.compute_intervals(tl)
[interval.human for interval in intervals]

["Timeline: from '2024-03-28 at 18:51:38' to '2024-03-28 at 18:58:23', duration=6 min 45 s",
 "Timeline: from '2024-03-28 at 20:29:03' to '2024-03-28 at 20:33:43', duration=4 min 40 s",
 "Timeline: from '2024-03-29 at 18:03:33' to '2024-03-29 at 18:10:18', duration=6 min 45 s",
 "Timeline: from '2024-03-29 at 19:40:38' to '2024-03-29 at 19:46:18', duration=5 min 40 s",
 "Timeline: from '2024-03-30 at 18:52:18' to '2024-03-30 at 18:58:33', duration=6 min 15 s",
 "Timeline: from '2024-03-31 at 18:03:58' to '2024-03-31 at 18:10:38', duration=6 min 40 s",
 "Timeline: from '2024-03-31 at 19:42:13' to '2024-03-31 at 19:44:48', duration=2 min 35 s",
 "Timeline: from '2024-04-01 at 18:53:03' to '2024-04-01 at 18:57:48', duration=4 min 45 s"]

NB: you can also compute *a posteriori* the intersection of two sets of timelines with `get_intersections_timelines`, but the other method is prefered.